<a href="https://colab.research.google.com/github/Mikelmillz/Project-4-Netflix-TV-Shows-and-Movies/blob/main/LaurasFolder/kearasModel_drama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Found this in https://keras.io/keras_tuner/ when looking for how to tune
!pip install keras-tuner --upgrade

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow import keras
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [3]:
# read the filmtitles to a pandas data frame
film_titles_df = pd.read_csv("filmtitles2.csv")
film_titles_df

,id,title,type,description,release_year,age_certification,runtime,production_countries,imdb_id,imdb_score,...,history,horror,music,reality,romance,scifi,sport,thriller,war,western
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,US,tt0075314,8.2,...,0,0,0,0,0,0,0,0,0,0
1,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,US,tt0068473,7.7,...,0,0,0,0,0,0,0,1,0,0
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,GB,tt0071853,8.2,...,0,0,0,0,0,0,0,0,0,0
3,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"GB,US",tt0061578,7.7,...,0,0,0,0,0,0,0,0,1,0
4,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,GB,tt0079470,8.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,tm1066324,Super Monsters: Once Upon a Rhyme,MOVIE,The Super Monsters rethink exemplary fantasies...,2021,NaN,25,NaN,tt14586752,5.6,...,0,0,0,0,0,0,0,0,0,0
3425,tm1097142,My Bride,MOVIE,The story follows a young man and woman who go...,2021,NaN,93,EG,tt14216488,5.0,...,0,0,0,0,1,0,0,0,0,0
3426,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,NG,tt13857480,6.8,...,0,0,0,0,1,0,0,0,0,0
3427,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,NaN,tt11803618,7.7,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Determine the number of unique values in each column.
film_titles_df.nunique()

id                      3429
title                   3411
type                       1
description             3424
release_year              62
age_certification          5
runtime                  187
production_countries     390
imdb_id                 3429
imdb_score                74
imdb_votes              2747
action                     2
animation                  2
comedy                     2
crime                      2
documentation              2
drama                      2
european                   2
family                     2
fantasy                    2
history                    2
horror                     2
music                      2
reality                    2
romance                    2
scifi                      2
sport                      2
thriller                   2
war                        2
western                    2
dtype: int64

In [5]:
# Drop the additional ID columns, 'id', 'title', 'description', 'imdb_id'
film_titles_df = film_titles_df.drop(columns=[ 'id', 'title', 'description', 'imdb_id', 'imdb_votes', 'type' ])
film_titles_df

,release_year,age_certification,runtime,production_countries,imdb_score,action,animation,comedy,crime,documentation,...,history,horror,music,reality,romance,scifi,sport,thriller,war,western
0,1976,R,114,US,8.2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1972,R,109,US,7.7,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1975,PG,91,GB,8.2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1967,NaN,150,"GB,US",7.7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1979,R,94,GB,8.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,2021,NaN,25,NaN,5.6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3425,2021,NaN,93,EG,5.0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3426,2021,NaN,100,NG,6.8,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3427,2021,NaN,134,NaN,7.7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Determine the number of unique values in each column.
film_titles_df.nunique()

release_year             62
age_certification         5
runtime                 187
production_countries    390
imdb_score               74
action                    2
animation                 2
comedy                    2
crime                     2
documentation             2
drama                     2
european                  2
family                    2
fantasy                   2
history                   2
horror                    2
music                     2
reality                   2
romance                   2
scifi                     2
sport                     2
thriller                  2
war                       2
western                   2
dtype: int64

In [7]:
# Determine the data type of each column.
film_titles_df.dtypes

release_year              int64
age_certification        object
runtime                   int64
production_countries     object
imdb_score              float64
action                    int64
animation                 int64
comedy                    int64
crime                     int64
documentation             int64
drama                     int64
european                  int64
family                    int64
fantasy                   int64
history                   int64
horror                    int64
music                     int64
reality                   int64
romance                   int64
scifi                     int64
sport                     int64
thriller                  int64
war                       int64
western                   int64
dtype: object

In [8]:
column_counts = {}
# Iterate over each column in the DataFrame
for column in film_titles_df.columns:
    # Calculate value counts for the current column
     column_counts[column] = (film_titles_df[column] == 1).sum()

# Print or use column counts as needed
for column, counts in column_counts.items():
    print(f"Counts for column '{column}':\n{counts}\n")

Counts for column 'release_year':
0

Counts for column 'age_certification':
0

Counts for column 'runtime':
0

Counts for column 'production_countries':
0

Counts for column 'imdb_score':
0

Counts for column 'action':
688

Counts for column 'animation':
229

Counts for column 'comedy':
1493

Counts for column 'crime':
533

Counts for column 'documentation':
539

Counts for column 'drama':
1805

Counts for column 'european':
329

Counts for column 'family':
328

Counts for column 'fantasy':
304

Counts for column 'history':
145

Counts for column 'horror':
249

Counts for column 'music':
173

Counts for column 'reality':
9

Counts for column 'romance':
683

Counts for column 'scifi':
200

Counts for column 'sport':
113

Counts for column 'thriller':
792

Counts for column 'war':
98

Counts for column 'western':
28



In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
film_titles_num_df = pd.get_dummies(film_titles_df)
display(film_titles_num_df.head())
display(film_titles_num_df.tail())

,release_year,runtime,imdb_score,action,animation,comedy,crime,documentation,drama,european,...,"production_countries_UY,MX,ES","production_countries_VE,AR",production_countries_VN,production_countries_XX,production_countries_ZA,"production_countries_ZA,AO","production_countries_ZA,GB","production_countries_ZA,GB,US","production_countries_ZA,US,CA",production_countries_ZW
0,1976,114,8.2,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1972,109,7.7,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1975,91,8.2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1967,150,7.7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1979,94,8.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,release_year,runtime,imdb_score,action,animation,comedy,crime,documentation,drama,european,...,"production_countries_UY,MX,ES","production_countries_VE,AR",production_countries_VN,production_countries_XX,production_countries_ZA,"production_countries_ZA,AO","production_countries_ZA,GB","production_countries_ZA,GB,US","production_countries_ZA,US,CA",production_countries_ZW
3424,2021,25,5.6,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3425,2021,93,5.0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3426,2021,100,6.8,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3427,2021,134,7.7,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3428,2021,90,3.8,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
 # Split our preprocessed data into our features and target arrays
y = film_titles_num_df["drama"].values
x = film_titles_num_df.drop(["drama"],axis=1).values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=10, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)
x_train_scaled

array([[ 0.29336381, -1.13934023,  1.09453953, ...,  0.        ,
        -0.01972575, -0.01972575],
       [ 0.54465096,  0.06549738, -0.98693078, ...,  0.        ,
        -0.01972575, -0.01972575],
       [ 0.67029453, -1.52914063, -1.68075422, ...,  0.        ,
        -0.01972575, -0.01972575],
       ...,
       [ 0.67029453, -0.07624822,  0.83435574, ...,  0.        ,
        -0.01972575, -0.01972575],
       [ 0.16772024,  1.3057714 ,  1.09453953, ...,  0.        ,
        -0.01972575, -0.01972575],
       [ 0.54465096, -1.91894104,  0.22726023, ...,  0.        ,
        -0.01972575, -0.01972575]])

In [12]:
# Write a function that creates and returns a Keras model.
def build_model(hp):
    nn_opt = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])


    nn_opt.add(tf.keras.layers.Dense(
        # tune the number of units
        units=hp.Int("units", min_value=1, max_value=5, step=2),
            activation=activation, input_dim=x_train_scaled.shape[1]))

     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):    # can go to 5
        nn_opt.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=5,   # play with this from 5
            step=2),
            activation=activation))

    nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # compile the model
    nn_opt.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_opt


In [13]:
 # Initialize a tuner (here, RandomSearch). We use objective to specify the objective to select the best models, and we use max_trials to specify the number of different models to try.
 tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2)

 tuner.search(x=x_train_scaled, y=y_train, epochs=20, validation_data=(x_test_scaled, y_test), batch_size=32)

Reloading Tuner from ./untitled_project/tuner0.json


In [14]:
# Start the search and get the best model:
best_model = tuner.get_best_models()[0]

# print the accuracy
print("Best accuracy:", best_model.evaluate(x_test_scaled, y_test)[1])

27/27 [==============================] - 0s 2ms/step - loss: 1.3862 - accuracy: 0.5035
Best accuracy: 0.503496527671814


In [15]:
# Evaluate the model using the test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

27/27 - 0s - loss: 1.3862 - accuracy: 0.5035 - 272ms/epoch - 10ms/step
Loss: 1.38618803024292, Accuracy: 0.503496527671814
